In [29]:
import pandas as pd
from yaml import safe_load
from pandas import json_normalize

from sentence_transformers import SentenceTransformer
from transformers import AutoModel, BertTokenizer
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import numpy as np
import torch
import tensorflow as tf

2023-08-07 13:27:18.590967: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [57]:
with open('../data/terms_definitions_final.txt', 'rb') as fp:
    documents = pickle.load(fp)

tokenizerMacBERTh = BertTokenizer.from_pretrained("emanjavacas/MacBERTh", max_length=512)
modelMacBERTh  = AutoModel.from_pretrained("emanjavacas/MacBERTh", max_length=512)
modelMacBERTh.max_seq_length = 509
modelMPNET = SentenceTransformer('all-mpnet-base-v2')
modelMPNET._first_module().max_seq_length = 509


Some weights of the model checkpoint at emanjavacas/MacBERTh were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [58]:
#Note: the data require more preprocessing before it can be embedding using MacBERTh. However, the tokenizer is working and this is the
# correct code for creating the embedding

tokens = []
for i in tqdm(range(len(documents)), desc="Tokenizing"):
    curr_token = tokenizerMacBERTh(documents[i], padding=True, truncation=True, return_tensors='pt')
    if(len(curr_token) <= 512):
        tokens.append(curr_token)
    else:
        tokens.append(curr_token[:512])
print("documents tokenized")

text_embeddings_MacBERTh = []
with torch.no_grad():   
    for i in tqdm(range(len(tokens)), desc="Embedding"):
        output = modelMacBERTh(**(tokens[i]))
        text_embeddings_MacBERTh.append(output.last_hidden_state.mean(dim=1))
print("documents embedded")

all_embeddings_total_MacBERTh  = np.stack([embedding.numpy() for embedding in text_embeddings_MacBERTh])
np.save('embeddings_summaries_total_ed_macberth.npy', all_embeddings_total_MacBERTh)
print("Macberth embeddings finished and saved")

Tokenizing: 100%|██████████| 23122/23122 [01:32<00:00, 249.98it/s]


documents tokenized


Embedding:   0%|          | 22/23122 [00:01<22:11, 17.35it/s]


RuntimeError: The size of tensor a (575) must match the size of tensor b (512) at non-singleton dimension 1

In [6]:
text_embeddings_MPNET = modelMPNET.encode(documents, show_progress_bar = True)
all_embeddings_total_MPNET = np.array(text_embeddings_MPNET)
np.save('embeddings_summaries_total_ed_mpnet.npy', all_embeddings_total_MPNET)
print("Mpnet embeddings finished and saved")

Batches: 100%|██████████| 723/723 [58:35<00:00,  4.86s/it]


Mpnet embeddings finished and saved
finished
